In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dotenv import load_dotenv
load_dotenv();

In [ ]:
import base64
import json
import logging
import os
import re
import smtplib
import sys
import time
import threading
import traceback
from functools import partial, wraps
from itertools import chain
from importlib import reload
from pathlib import Path
from uuid import uuid4
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from datetime import timezone

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import schedule  
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser
from pytz import timezone


from api.watchdog import specs
from api.watchdog.specs import get_trades, get_targets
from api.watchdog.pricer import Pricer
from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root
)
from gioutils.tools_clarion import get_history_plotter_book

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [ ]:
from api.jobs.risk import daily_var_email
from api.jobs.risk_intraday import var_reduction_email, var_reduction_email2
from api.jobs.macd_watcher import macd_signals_email
from api.jobs.positions import history_plotter_email

In [ ]:
tz = timezone('UTC')

In [ ]:
def catch_exceptions(cancel_on_failure=False):
    def catch_exceptions_decorator(job_func):
        @wraps(job_func)
        def wrapper(*args, **kwargs):
            try:
                return job_func(*args, **kwargs)
            except:
                print(traceback.format_exc())
                if cancel_on_failure:
                    return schedule.CancelJob
        return wrapper
    return catch_exceptions_decorator

In [ ]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

In [ ]:
super_clarion(clarion)
runner = Runner(clarion)

In [ ]:
schedule.clear()

In [ ]:
daily_var_email_job = schedule.every().day.at('08:00').do(catch_exceptions()(daily_var_email))
# var_reduction_email_jobs = [schedule.every().day.at(f"{i:02d}:00").do(catch_exceptions()(var_reduction_email), clarion=clarion) for i in range(8, 21)]
var_reduction_email_jobs2 = [schedule.every().day.at(f"{i:02d}:00").do(catch_exceptions()(var_reduction_email2), clarion=clarion, bq=bq, send_email=True) for i in range(8, 21)]
# var_reduction_email_job  = schedule.every(1).hours.do(catch_exceptions()(var_reduction_email), clarion=clarion)
macd_signals_email_job = schedule.every().day.at('16:30').do(catch_exceptions()(macd_signals_email), bq=bq)

In [ ]:
while True:
    schedule.run_pending()
    time.sleep(1)

In [ ]:
while True:
    try:
        schedule.run_pending()
    except: 
        pass
    time.sleep(1)

In [ ]:
schedule.get_jobs()